# Early Model For Testing (Documents + Vocabs)

In [4]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# .env 파일 불러오기
load_dotenv()

# 환경 변수에서 값 불러오기
embedding_endpoint = os.getenv("Embedding_ENDPOINT")
embedding_api_key = os.getenv("Embedding_API_KEY")

openai_endpoint = os.getenv("OPENAI_ENDPOINT")
openai_api_key = os.getenv("OPENAI_API_KEY")

search_endpoint = "https://6a021-aisearch.search.windows.net"
search_key = os.getenv("AI_Search_API_KEY")
search_index = os.getenv("AI_Search_INDEX_NAME")

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=embedding_endpoint,
    api_key=openai_api_key,
    api_version="2024-05-01-preview"
)

# system 프롬프트 설정
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "너는 용어에 대해 물어보면 친절히 답해주는 도우미야."
            }
        ]
    },
    {
        "role": "user",
        "content": "가산하다가 뭐야?"
    }
]

# Completion 요청
completion = client.chat.completions.create(
    model="teaam7-gpt4o",  # 배포 이름
    messages=chat_prompt,
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stream=False,
    extra_body={
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": "default",
                    "query_type": "vector_semantic_hybrid",
                    "fields_mapping": {
                        "content_fields_separator": "\n",
                        "content_fields": ["content"],
                        "filepath_field": "filepath",
                        "title_field": "title",
                        "url_field": "url",
                        "vector_fields": ["contentVector"]
                    },
                    "in_scope": True,
                    "role_information": "너는 용어에 대해 물어보면 친절히 답해주는 도우미야.",
                    "filter": None,
                    "strictness": 3,
                    "top_n_documents": 6,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    },
                    "embedding_dependency": {
                        "type": "deployment_name",
                        "deployment_name": "text-embedding-3-small"
                    }
                }
            }
        ]
    }
)

# 결과 출력
print(completion.choices[0].message.content)


"가산하다"는 "더하다" 또는 "추가하다"라는 뜻입니다. 예를 들어, **"공사에 납부하는 보증금으로서 주채무 원금에 가산되는 대출금"**이라는 문장에서 "가산되다"는 대출금이 원금에 더해진다는 의미로 사용됩니다 [doc2].


In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 로드
load_dotenv()

# 환경 변수 불러오기
embedding_endpoint = os.getenv("Embedding_ENDPOINT")
embedding_api_key = os.getenv("Embedding_API_KEY")

openai_api_key = os.getenv("OPENAI_API_KEY")
search_endpoint = "https://6a021-aisearch.search.windows.net"
search_key = os.getenv("AI_Search_API_KEY")
search_index = os.getenv("AI_Search_INDEX_NAME", "vocab")

# OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=embedding_endpoint,
    api_key=openai_api_key,
    api_version="2024-05-01-preview"
)

# ✅ Query rewrite 함수
def rewrite_query(user_input: str) -> str:
    prompt = [
        {"role": "system", "content": "사용자의 질문을 더 구체적이고 자연스럽게 바꿔주는 역할이야."},
        {"role": "user", "content": f"질문: {user_input}\n\n검색에 적합하도록 자연스럽게 다시 표현해줘."}
    ]

    response = client.chat.completions.create(
        model="teaam7-gpt4o",
        messages=prompt,
        max_tokens=100,
        temperature=0.7,
        stream=False
    )
    rewritten = response.choices[0].message.content.strip()
    return rewritten


# ✅ 메인 질문 함수
def ask_query(user_question: str) -> str:
    refined_question = rewrite_query(user_question)

    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "너는 용어에 대해 물어보면 친절히 답해주는 도우미야."
                }
            ]
        },
        {
            "role": "user",
            "content": refined_question
        }
    ]

    completion = client.chat.completions.create(
        model="teaam7-gpt4o",
        messages=messages,
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        stream=False,
        extra_body={
            "data_sources": [
                {
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "index_name": search_index,
                        "semantic_configuration": "default",
                        "query_type": "vector_semantic_hybrid",
                        "fields_mapping": {
                            "content_fields_separator": "\n",
                            "content_fields": ["content"],
                            "filepath_field": "filepath",
                            "title_field": "title",
                            "url_field": "url",
                            "vector_fields": ["contentVector"]
                        },
                        "in_scope": True,
                        "role_information": "너는 용어에 대해 물어보면 친절히 답해주는 도우미야.",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 6,
                        "authentication": {
                            "type": "api_key",
                            "key": search_key
                        },
                        "embedding_dependency": {
                            "type": "deployment_name",
                            "deployment_name": "text-embedding-3-small"
                        }
                    }
                }
            ]
        }
    )

    return completion.choices[0].message.content


# ✅ 예시 실행
if __name__ == "__main__":
    user_question = input("궁금한 걸 입력하세요: ")
    answer = ask_query(user_question)
    print("\n[답변]\n", answer)



[답변]
 담보대출이란 대출을 받을 때, 차주가 자신의 자산(예: 부동산, 자동차 등)을 담보로 제공하여 대출금을 빌리는 금융 거래를 의미합니다. 대출을 제공하는 금융기관은 차주가 제공한 담보를 통해 대출금을 회수할 수 있는 안전장치를 가지게 됩니다. 

예를 들어, 부동산 담보대출의 경우 주택이나 부동산을 담보로 대출을 받을 수 있으며, 대출금을 갚지 못할 경우 금융기관이 해당 담보 자산을 처분해 대출금을 회수할 수 있습니다. 이는 차주의 신용도와는 별개로 담보 자산의 가치에 따라 대출 가능 금액이 결정되는 경우가 많습니다.

또한, 담보대출은 통상적으로 신용대출보다 금리가 낮은 경우가 많으며, 담보 자산의 가치 평가와 대출 조건에 따라 변동이 있을 수 있습니다.
